<a href="https://colab.research.google.com/github/arewelearningyet/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
# prior belief about drunk driving in the population is 1/1000
prob_drunk_prior=1/1000
# false positive rate
false_positive_rate=0.08

p_drunk=prob_drunk_prior
p_not_drunk=1-p_drunk
true_positive_rate=1

In [29]:
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate):
  true_positive_rate=1
  p_drunk=prob_drunk_prior
  p_not_drunk=1-p_drunk
  return (true_positive_rate*p_drunk)/((true_positive_rate*p_drunk)+(false_positive_rate*p_not_drunk))
print('the probability that someone is drunk after one positive breathalyzer tests is', 
      prob_drunk_given_positive(prob_drunk_prior, false_positive_rate))

the probability that someone is drunk after one positive breathalizer tests is 0.012357884330202669


In [33]:
print('the probability that someone is drunk after two positive brethalyzer tests is',
      prob_drunk_given_positive
      (prob_drunk_given_positive
       (prob_drunk_prior, false_positive_rate), false_positive_rate))

the probability that someone is drunk after two positive brethalyzer tests is 0.13525210993291495


In [34]:
print(prob_drunk_given_positive
      (prob_drunk_given_positive
       (prob_drunk_given_positive
        (prob_drunk_given_positive
         (prob_drunk_prior, false_positive_rate), false_positive_rate), false_positive_rate), false_positive_rate))
print('four positive breathalyzer tests are needed in order to have a probability that\'s greater than 95% that a person is drunk beyond the legal limit')

0.9606895076105054
four positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit


In [0]:
# in a function that loops until >0.95 threshold is reached...
# defining with given parameters
def prob_drunk_given_pos(prob_drunk_prior, false_pos_rate, true_pos_rate):
  p_drunk=prob_drunk_prior # establish baseline prior assumptions
  p_not_drunk=1-p_drunk 
  # applying theorem
  posterior_probability=(
      true_pos_rate*p_drunk)/((true_pos_rate*p_drunk)+(false_positive_rate*p_not_drunk)
  )
  print(posterior_probability) # print initial calculated probability
  #establish while loop to continue while probability remains below threshold
  while posterior_probability<=0.95:
    p_drunk=posterior_probability # integrate new probability into calculations
    p_not_drunk=1-p_drunk
    # reapply theorem
    posterior_probability=(
        true_pos_rate*p_drunk)/((true_pos_rate*p_drunk)+(
            false_positive_rate*p_not_drunk)
  )
    print(posterior_probability) # print latest probability within threshold
    if posterior_probability ==p_drunk: # conditions that terminate while loop
      break

In [39]:
# run looping function to count iterations until threshold is reached
prob_drunk_given_pos((1/1000), .08, 1)

0.012357884330202669
0.13525210993291495
0.6615996951348605
0.9606895076105054


## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

2) Explore scipy.stats.bayes_mvs
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.

Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
Matplotlib Error Bars
Seaborn barplot with error bars
Vertical ines to show bounds of confidence interval
Confidence Intervals on Box Plots

Frequentist is exclusive of anything but explicit given parameters within a sample; Bayesian integrates prior assumptions, as well as integrating further measurements into calculations as they may progress.

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!